In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from fastai.structured import *
from fastai.column_data import *
import pandas as pd
import keras
import utils
from keras.models import model_from_yaml
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 20
import numpy as np
from pandas_summary import DataFrameSummary
from importlib import reload
reload(utils)

np.set_printoptions(threshold=50, edgeitems=20)

PATH='./data/'

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train_feats = pd.read_feather(r'./data/agg_ts_feas_p2017-01-30_e2017-03-12_s2016-01-01').drop('index', axis=1, errors='ignore')
val_feats  = pd.read_feather(r'./data/period_e2017-03-12_n42').drop('index', axis=1, errors='ignore')

In [ ]:
predictors = [f for f in train_feats.columns if f not in (['id','store_id','visit_date','end_date','air_area_name','visitors','month'])]

In [ ]:
import lightgbm as lgb

params = {
    'learning_rate': 0.1,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'min_data': 100,
    'min_hessian': 1,

#     'verbose': -1,
}


In [ ]:
lgb_train = lgb.Dataset(train_feats[predictors], train_feats['visitors'])
lgb_val = lgb.Dataset(val_feats[predictors], val_feats['visitors'])
# lgb_test = lgb.Dataset(test_feat[predictors], test_feat['visitors'])

In [ ]:
evals_result = {}
gbm = lgb.train(params,
                lgb_train,
                100,
                valid_sets=(lgb_train, lgb_val), # , lgb_val
                evals_result=evals_result,)

In [ ]:
print('Plot metrics during training...')
ax = lgb.plot_metric(evals_result)
plt.show()

## RRVF 1st place feats + deep learning

In [3]:
test_feats  = pd.read_feather(r'data/period_e2017-04-23_n39').drop('index', axis=1, errors='ignore')

In [4]:
cat_vars = [ 'store_id', 'air_area_name', 'month',  'dow', 'year', 'air_area_name0', 'air_genre_name']

In [8]:
contin_vars = [c for c in trn_f.columns if c not in [ 'store_id', 'air_area_name', 'month',  'dow', 'year', 'air_area_name0', 'air_genre_name', 'visit_date']]

In [7]:
trn_f = train_feats.drop(columns=['end_date'])
val_f = val_feats.drop(columns=['end_date'])
test_f = val_feats.drop(columns=['end_date'])

In [9]:
joined = pd.concat([trn_f, val_f])
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [10]:
joined = joined.drop(columns='visit_date')
test_f = test_f.drop(columns='visit_date')

In [62]:
# joined['visit_date'] = pd.to_datetime(joined.visit_date)
# test_f['visit_date'] = pd.to_datetime(test_f.visit_date)

In [11]:
apply_cats(test_f, joined)

In [12]:
for v in contin_vars:
    if v != 'id':
        joined[v] = joined[v].astype('float32')
        test_f[v] = test_f[v].astype('float32')

In [17]:
joined = joined.reset_index()

In [18]:
joined = joined.drop(columns='id')

In [19]:
subset_index = joined[:20]

In [ ]:
df, y, nas, mapper = proc_df(joined, 'visitors', do_scale=True)

In [ ]:
test_f = test_f.set_index("id")
df_test, _, nas, mapper = proc_df(test_f, 'visitors', do_scale=True, mapper=mapper, na_dict=nas)

In [ ]:
y_trn = train_feats.visitors.values
y_val = val_feats.visitors.values

In [ ]:
def inv_y(a): return np.exp(a) - 1

def rmsle(y_pred, targ):
    log_vars = targ - y_pred
    return math.sqrt(np.square(log_vars).mean())

max_log_y = np.max(y))
y_range = (0, max_log_y*1.2)

In [ ]:
md = ColumnarModelData.from_data_frames(PATH, trn_f, val_f, y_trn.astype(np.float32), y_val.astype(np.float32), cat_flds=cat_vars, bs=32, test_df=test_f)

In [ ]:
joined = pd.concat([trn_f, val_f])
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [ ]:
cat_sz = [(c, len(joined[c].cat.categories)+1) for c in cat_vars]
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
emb_szs

In [ ]:
m = md.get_learner(emb_szs, len(trn_f.columns)-len(cat_vars),
                   0.04, 1, [1000, 500], [0.001, 0.01], y_range=y_range, use_bn=True)

In [ ]:
lr = 1e-3

In [ ]:
m.lr_find()

## RRVF 1st place feats + Jeremy feats + deep learning